### Runtime setup
The following envs enable stable retries and quiet streaming.

- `SCILLM_FORCE_HTTPX_STREAM=1`
- `LITELLM_MAX_RETRIES=3`, `LITELLM_RETRY_AFTER=1`, `LITELLM_TIMEOUT=45`
- Requires `tenacity` installed for backoff.

In [1]:
import os
os.environ.setdefault('SCILLM_FORCE_HTTPX_STREAM','1')
os.environ.setdefault('LITELLM_MAX_RETRIES','3')
os.environ.setdefault('LITELLM_RETRY_AFTER','1')
os.environ.setdefault('LITELLM_TIMEOUT','45')
try:
    import tenacity  # noqa: F401
    print('tenacity: ok')
except Exception:
    print('tenacity missing — run: pip install tenacity')


tenacity: ok


# Router.parallel_acompletions

In [2]:
import os, asyncio
import nest_asyncio; nest_asyncio.apply()
from scillm import Router
router = Router(default_litellm_params={
  'api_base': os.environ['CHUTES_API_BASE'],
  'api_key': None,
  'custom_llm_provider': 'openai_like',
  'extra_headers': {'Authorization': f"Bearer {os.environ['CHUTES_API_KEY']}"},
})
prompts = ['OK-A','OK-B','OK-C']
reqs = [{
  'model': os.environ['CHUTES_MODEL'],
  'messages': [{'role':'user','content': p}],
  'kwargs': {'max_tokens': 8, 'temperature': 0, 'timeout': 20}
} for p in prompts]
async def run():
  outs = await router.parallel_acompletions(requests=reqs, concurrency=2)
  print([ (o.get('choices',[{}])[0].get('message',{}).get('content','') or '').strip() for o in outs ])
loop = asyncio.get_event_loop()
loop.run_until_complete(run())

['', '', '']
